**Testing ionospheric model NeQuick 2 using Swarm satellite measurements of electron density**


---
Web: https://gitlab.com/BoryMAndy/

Author: Boris Matiushin

In [ ]:
! pip install basemap

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import csv
import os
import warnings
import matplotlib
import datetime as dt
import h5py
import zipfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from mpl_toolkits.basemap import Basemap
from tqdm.notebook import tqdm


warnings.filterwarnings('ignore')
matplotlib.rc('text', usetex = False)

Global variables

In [ ]:
DPI = 650

KP_CRIT = 45
AE_CRIT = 500

PATH_TO_FILE = {'A': 'SWARM_A_fix.txt',
                'B': 'SWARM_B_fix.txt',
                'C': 'SWARM_C_fix.txt'}

# Functions

In [ ]:
# Read file and convert variables
def read_res_file(filepath):
  data = pd.read_csv(filepath,
                     sep = ' ',
                     usecols = [0, 1, 3, 4, 5, 6],
                     names = ['date', 'time', 'lat', 'lon', 'swarm', 'nq'],
                     dtype = {'date': str, 'time': str, 'lat': float, 'lon': float, 'swarm': float, 'nq': float}
                     )

  data['swarm'] = data['swarm'].rolling(window=31, center=True).mean()
  data = data[::31]

  # dates converting
  data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'], format = '%d-%m-%y %H:%M:%S.%f')
  data['date'] = pd.to_datetime(data['date'], format = '%d-%m-%y')
  # dencity to cubic meteres
  data['swarm'] = data['swarm'] * 1000000

  data['err_abs'] = (data['nq'] - data['swarm']) / 10**12 # absolute error
  data['err_otn'] = 100 * (data['nq'] - data['swarm']) / data['swarm'] # relative error

  return data

In [ ]:
# reading 2 files for 1 hight
def read_AC():
  data_A = read_res_file(PATH_TO_FILE['A'])
  data_A.to_csv('data_A.csv', index=False)
  data_A = 0

  data_C = read_res_file(PATH_TO_FILE['C'])
  data_A = pd.read_csv('data_A.csv')

  data_A['date'] = pd.to_datetime(data_A['date'])
  data_A['datetime'] = pd.to_datetime(data_A['datetime'])

  data_AC = pd.concat([data_A,
                  data_C
                  ])
  data_A = data_C = 0

  return data_AC

Next function make net of average values, that receives the format for the coordinate grid, satellite data, dates for processing and produces a matrix for drawing the map. You can also specify which month you want to plot. If the *month* parameter is 0, the annual average is taken.

In [ ]:
# Make net of averege values
def map_fill(data_loc, dates_loc, grip='full', module_val=True, month=0, err = 'err_otn'):

  if grip == 'full':
    lats = np.arange(-90, 91, 2)
    lons = np.arange(-180, 181, 6)
    lon, lat = np.meshgrid(lons, lats) # net
  elif grip == 'pol':
    lats = np.arange(50, 91, 2)
    lons = np.arange(-180, 181, 6)
    lon, lat = np.meshgrid(lons, lats) # net
  else:
    raise NameError('requested format is not defined')


  print('dates_processing')
  # select required month if needed
  if month != 0:
    dates_loc_mth = []
    dates_dt = pd.to_datetime(dates_loc)
    for day in dates_dt:
      if day.month == month:
        dates_loc_mth.append(day)
    dates_loc_mth = list(map(np.datetime64, dates_loc_mth))

    if len(dates_loc_mth) == 0:
      return 'code 0'

    prob = data_loc.loc[data_loc['date'].isin(dates_loc_mth)][[err, 'lat', 'lon', 'swarm']]

  else:
    prob = data_loc.loc[data_loc['date'].isin(dates_loc)][[err, 'lat', 'lon', 'swarm']]


  map_filled = []
  print('data_processing')
  for la in tqdm(lats):
    ad = []

    for lo in lons:

      max_val = np.abs(prob[err]) < 1000
      min_val = prob['swarm'] != 0
      lat_ge = prob['lat'] >= la - 1
      lat_le = prob['lat'] <= la + 1
      lon_ge = prob['lon'] >= lo - 3
      lon_le = prob['lon'] <= lo + 3

      series = prob.loc[lat_ge & lat_le & lon_ge & lon_le
                        & max_val & min_val
                        ][err] # selecting a spatial square

      mean_err = np.nan
      if len(series) > 1: # min counts in located square to write result
        if module_val == True: # take absolute values depending on parameter
          mean_err = np.mean(np.abs(series))
        elif module_val == False:
          mean_err = np.mean(series)

      ad.append(mean_err)


    map_filled.append(ad)
  return map_filled

The following function draws a map of the entire surface of the earth or pole depending on the parameter *grip*.

In [ ]:
def plot_full_map(map_filled_loc, grip='full', name='something_nameless.png'):
  if map_filled_loc == 'code 0':
    return 'There is no data selected'

  if grip == 'full':
    proj = 'cyl'
    lats = np.arange(-90, 91, 2)
    lons = np.arange(-180, 181, 6)
    lon, lat = np.meshgrid(lons, lats) # net

    map_earth = Basemap(projection=proj,
                        llcrnrlat=-90,
                        urcrnrlat=90,
                        llcrnrlon=-180,
                        urcrnrlon=180,
                        resolution='c')
    x, y = map_earth(lon, lat)
    fig = plt.figure(figsize = (10, 8))
    map_earth.fillcontinents()
    map_earth.drawcoastlines()
    grey = plt.pcolormesh(x, y,
                          map_filled_loc,
                          alpha = 0.9,
                          cmap = 'binary'
                          )

    parallels = np.arange(-90.,91,30.)
    # labels = [left,right,top,bottom]
    map_earth.drawparallels(parallels,
                            labels = [True, True, True, True],
                            fontsize = 24,
                            #labelstyle='+/-'
                            )
    meridians = np.arange(0., 361., 60.)
    map_earth.drawmeridians(meridians,
                            labels = [True, True, True, True],
                            fontsize = 24,
                            #labelstyle='+/-'
                            )
    # tick = np.linspace(-0.6, 0.6, 5, endpoint=True)
    clrb = plt.colorbar(grey, orientation="horizontal", fraction=0.046, pad=0.07)
    # clrb.set_ticks(np.arange(50, 176, 25)) # colorbar ticks
    clrb.ax.tick_params(labelsize=20)
    clrb.set_label('$\it{\Delta}$, %', fontsize=24)


  elif grip == 'pol':
    proj = 'npstere'
    lats = np.arange(50, 91, 2)
    lons = np.arange(-180, 181, 6)
    lon, lat = np.meshgrid(lons, lats) # net

    map_earth = Basemap(projection=proj,
                        boundinglat = 60,
                        lon_0 = 0,
                        resolution = 'l')
    x, y = map_earth(lon, lat)
    fig = plt.figure(figsize = (10, 8))
    map_earth.fillcontinents()
    map_earth.drawcoastlines()
    plt.pcolormesh(x, y, map_filled_loc, alpha = 0.9, cmap = 'binary')

    parallels = np.arange(0.,81,10.)
    # labels = [left,right,top,bottom]
    map_earth.drawparallels(parallels,
                            labels = [False, False, True, False],
                            fontsize = 24,
                            #labelstyle='+/-'
                            )
    meridians = np.arange(10., 351., 20.)
    map_earth.drawmeridians(meridians,
                            labels = [True, False, False, True],
                            fontsize = 24,
                            #labelstyle='+/-'
                            )
    clrb = plt.colorbar(format = '%.f')
    #clrb.set_ticks(np.arange(30, 51, 5)) # colorbar ticks
    clrb.ax.tick_params(labelsize=20)
    clrb.set_label('$\it{\Delta}$, %', fontsize=24)

  plt.savefig(name, dpi=DPI)

The following function calculates the average errors in the latitudinal zone from lat0 to lat1, under the klik parameter one can specify geomagnetic conditions for calculation.

In [ ]:
def err_lat(datal, lat0=0, lat1=90, klik='storm', module_val=True, err='err_otn'):

  min_val = datal['swarm'] != 0

  if klik == 'storm':
    prob = datal.loc[datal['date'].isin(date64_storm) & min_val][[err, 'lat']] # select corresponding dates

    if module_val == True: # take absolute values depending on parameter
      err_abs = np.abs(prob.loc[(prob['lat'] >= lat0) & (prob['lat'] <= lat1)][err])
    else:
      err_abs = prob.loc[(prob['lat'] >= lat0) & (prob['lat'] <= lat1)][err]

    meanerr = np.mean(err_abs)
    stderr = np.std(err_abs)


  elif klik == 'calm':
    prob = datal.loc[datal['date'].isin(date64_calm) & min_val][[err, 'lat']] # select corresponding dates

    if module_val == True: # take absolute values depending on parameter
      err_abs = np.abs(prob.loc[(prob['lat'] >= lat0) & (prob['lat'] <= lat1)][err])
    else:
      err_abs = prob.loc[(prob['lat'] >= lat0) & (prob['lat'] <= lat1)][err]

    meanerr = np.mean(err_abs)
    stderr = np.std(err_abs)

  else:
    raise print('klik = "calm" or "storm", there are no other options')


  return {'error': f"{meanerr:0.3f}+-{stderr:0.3f}",
          'cardo': len(err_abs)} # capacity

Monthly average errors plot for 2014 year northern hemisphere

In [ ]:
def plot_mnth(datal, err='err_otn', name='slide.png'):
  win_size = 20000 # depends on measurements per month

  fig, ax = plt.subplots(nrows=3, ncols=1, sharex=True)
  names = ['low lat', 'mid lat', 'high lat']
  for i, latitude in enumerate([0, 30, 60]):

      prob = datal.loc[(datal['lat']>=latitude) & (datal['lat']<=latitude+30) & (datal['swarm'] != 0)][[err, 'datetime']]
      prob[err] = np.abs(prob[err])
      prob['std'] = prob[err].rolling(window=win_size, center=True).std()
      prob[err] = prob[err].rolling(window=win_size, center=True).mean()

      prob_frame = pd.DataFrame({'time': prob['datetime'], 'err': prob[err], 'std': prob['std']})
      ax[i].plot(prob['datetime'], prob[err], color='black')
      ax[i].fill_between(prob['datetime'], prob[err] - prob['std'], prob[err] + prob['std'], color='grey')
      ax[i].set_title(names[i])
      ax[i].set_ylim(auto=True)
      ax[i].set_ylabel('$\it{\Delta}$, %', fontsize=12)
      ax[i].grid()

  myFmt = mdates.DateFormatter('%m') # dates format
  ax[i].xaxis.set_major_formatter(myFmt)
  ax[i].set_xlabel('date, month')
  ax[i].set_xlim(left=pd.to_datetime('01-01-2014'),
                 right=pd.to_datetime('31-12-2014'))

  plt.tight_layout()
  plt.savefig(name, dpi=DPI)

# Indexes of geomagnetic activity

Plot Kp, f107 and AE indexes for entire year


In [ ]:
with open('geoindexes_lowlat.dat') as file_kp:
  kp = []
  time_kp = []

  i = 0
  for row in file_kp:
    if i == 0:
      i += 1
      continue

    time_kp.append(dt.datetime.strptime(row.split()[0] + ' ' + row.split()[1], "%d-%m-%Y %H:%M").date())
    kp.append(float(row.split()[2]))

with open('geoindexes_highlat.txt') as file_ae:
  ae = []
  time_ae = []

  i = 0
  for row in file_ae:
    if i == 0:
      i += 1
      continue

    time_ae.append(dt.datetime.strptime(row.split()[0] + ' ' + row.split()[1], "%d/%m/%Y %H:%M").date())
    ae.append(float(row.split()[2]))

f107_file = csv.DictReader(open('cls_radio_flux_f107(2014).csv'))
time_f107 = []
idx_f107 = []
for string in f107_file:
  idx_f107.append(float(string.get('absolute_f107 (solar flux unit (SFU))')))
  time_f107.append(dt.datetime.strptime(string.get('time (yyyy MM dd)'), "%Y %m %d").date())

fig, ax = plt.subplots(nrows=3, ncols=1, sharex=True)
ax[0].plot(time_kp, kp, color='black')
ax[1].plot(time_f107, idx_f107, color='black')
ax[2].plot(time_ae, ae, color='black')
ax[0].hlines(y=KP_CRIT_DS,
             xmin=time_kp[0],
             xmax=time_kp[-1],
             color='grey',
             linestyle='dashdot',
             linewidth=3
             )
ax[2].hlines(y=AE_CRIT,
             xmin=time_ae[0],
             xmax=time_ae[-1],
             color='grey',
             linestyle='dashdot',
             linewidth=3
             )
ax[0].grid()
ax[1].grid()
ax[2].grid()

ax[2].set_xticklabels(time_ae, rotation=45) # labels rotation

for i in range(3): # ticks size
  for label in (ax[i].get_xticklabels() + ax[i].get_yticklabels()):
    label.set_fontsize(13)


ax[0].set_ylabel('$\it{Kp}$', fontsize=13)
ax[2].set_xlabel('Date, month.year', fontsize=13)
ax[1].set_ylabel('$\it{F10.7}$, sfu', fontsize=13)
ax[2].set_ylabel('$\it{AE}$, nT', fontsize=13)

myFmt = mdates.DateFormatter('%m.%Y') # date format
ax[0].xaxis.set_major_formatter(myFmt)

plt.tight_layout()
plt.savefig('indexes.png', dpi=DPI)

# Division of dates into active (storm) and quiet (calm)

Find active dates for Kp and AE indexes


In [ ]:
with open('geoindexes_lowlat.dat') as file_kp:
  date_kp = []

  i = 0
  for row in file_kp:
    if i == 0:
      i += 1
      continue
    if float(row.split()[2]) >= KP_CRIT:
      date_kp.append(dt.datetime.strptime(row.split()[0] + ' ' + row.split()[1], "%d-%m-%Y %H:%M").date())

  date_kp = set(date_kp)

In [ ]:
with open('geoindexes_highlat.txt') as file_ae:
  date_ae = []
  ae = []

  i = 0
  for row in file_ae:
    if i == 0:
      i += 1
      continue

    if float(row.split()[2]) >= AE_CRIT:
      date_ae.append(dt.datetime.strptime(row.split()[0] + ' ' + row.split()[1], "%d/%m/%Y %H:%M").date())

  date_ae = set(date_ae)

In [ ]:
cross_date = date_kp.intersection(date_ae)

In [ ]:
date64_storm = [] # format converting
for _ in date_kp:
  date64_storm.append(np.datetime64(_))

In [ ]:
begin = dt.datetime.strptime('2014-01-01', '%Y-%m-%d').date()
day = dt.timedelta(days = 1)
date64_calm = []


for i in range(0, 365, 1): # collect calm days of the year
  date_temp = np.datetime64((begin + i * day))

  if date_temp not in date64_storm:
    date64_calm.append(date_temp)

# Error maps (460 km)

In [ ]:
data_AC = read_AC()

storm days:

In [ ]:
map_filled = map_fill(data_AC, date64_storm, grip='full', module_val=True, err='err_otn', month=0)
plot_full_map(map_filled, 'full', 'map_storm_460.png')

calm days:

In [ ]:
plot_full_map(map_fill(data_AC, date64_calm, grip='full', module_val=True, err='err_otn', month=0),
              'full',
              'map_calm_460.png')

Pole maps:

In [ ]:
plot_full_map(map_fill(data_AC, date64_calm, grip='pol', module_val=True, err='err_otn', month=0), 'pol', 'map_calm_460_pol.png')

In [ ]:
plot_full_map(map_fill(data_AC, date64_storm, grip='pol', module_val=True, err='err_otn', month=0), 'pol', 'map_storm_460_pol.png')

In [ ]:
data_AC = 0

# Error maps (530 km):

In [ ]:
data_B = read_res_file(PATH_TO_FILE['B'])

Plot map for each month

In [ ]:
for m in tqdm(range(1, 13, 1)):
  plot_full_map(map_fill(data_B, date64_calm, grip='full', module_val=True, month=m, err='err_otn'), 'full', 'map_calm_530 ' + str(m) + '.png')

Full map for entire year (calm days)

In [ ]:
plot_full_map(map_fill(data_B, date64_calm, grip='full', module_val=True, month=0), 'full', 'map_calm_530.png')

Full map for entire year (storm days)

In [ ]:
plot_full_map(map_fill(data_B, date64_storm, grip='full', module_val=True, month=0), 'full', 'map_storm_530.png')

Pole maps:

In [ ]:
plot_full_map(map_fill(data_B, date64_calm, grip='pol', module_val=True, month=0), 'pol', 'map_calm_530_pol.png')

In [ ]:
plot_full_map(map_fill(data_B, date64_storm, grip='pol', module_val=True, month=0), 'pol', 'map_strom_530_pol.png')

# Average annual errors for different latitudinal zones (460 km)

In [ ]:
data_AC = read_AC()

Calculate MAE and std for storm and calm periods and different zones.

In [ ]:
condition = ['calm',
             'storm']

for k in condition:
  print(k)
  for latitude in tqdm([#-90, -60, -30,
                        0, 30, 60]):
    err_res = err_lat(data_AC, lat0=latitude, lat1=latitude + 30, klik=k, module_val=True, err='err_otn')

    print(f"mean error {k} {latitude}-{latitude+30}: {err_res['error']} capacity: {err_res['cardo']}")

In [ ]:
data_AC = 0 # clean RAM

# Average annual errors for different latitudinal zones (530 km)

In [ ]:
data_B = read_res_file(PATH_TO_FILE['B'])

In [ ]:
condition = ['calm', 'storm']

for k in condition:
  for latitude in tqdm([#-90, -60, -30,
                        0, 30, 60]):
    err_res = err_lat(data_B, lat0=latitude, lat1=latitude+30, klik=k, module_val=True, err='err_otn')

    print(f"mean error {k} {latitude}-{latitude+30}: {err_res['error']} capacity: {err_res['cardo']}")

In [ ]:
data_B = 0 # clean RAM

# Rolling monthly errors

In [ ]:
data_B = read_res_file(PATH_TO_FILE['B']) # 530 km

In [ ]:
plot_mnth(data_B, name='slide_530.png')

In [ ]:
data_B = 0 # clean RAM

In [ ]:
data_AC = read_AC()

In [ ]:
data_AC = data_AC.sort_values('datetime') # due to two sequential datasets

In [ ]:
plot_mnth(data_AC, name='slide_460.png')

In [ ]:
data_AC = 0 # clean RAM